In [1]:
import pathlib
import pandas as pd
import numpy as np

In [6]:
data_dir = pathlib.Path().absolute().parent.as_posix() + '/data/raw/train.csv'

df = pd.read_csv(data_dir)

In [15]:
df.sample(3)

,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Order_Date,Time_Orderd,Time_Order_picked,Weatherconditions,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,Time_taken(min)
29002,0xc1be,AGRRES12DEL03,29,4.6,27.165108,78.015053,27.195108,78.045053,11-02-2022,20:35:00,20:40:00,conditions Fog,Jam,1,Drinks,scooter,1,No,Metropolitian,(min) 27
43806,0x9d27,HYDRES20DEL02,39,4,17.483216,78.552111,17.533216,78.602111,24-03-2022,20:35:00,20:50:00,conditions Stormy,Jam,0,Meal,motorcycle,0,No,Urban,(min) 49
5588,0x92db,JAPRES03DEL02,21,4.2,26.913483,75.803139,26.933483,75.823139,13-03-2022,09:10:00,09:20:00,conditions Stormy,Low,0,Drinks,motorcycle,1,No,NaN,(min) 20


In [ ]:
# drop ID & Delivery_pperson_id

<font color = 'orange'>

#### <b> City
- handle NaN values
- handle extra space at suffix
- change data type

In [66]:
df['City'].unique()

array(['Urban ', 'Metropolitian ', 'Semi-Urban ', 'NaN '], dtype=object)

In [67]:
df[df['City'] == 'NaN '].shape[0]

1200

<font color = 'orange'>

#### <b> Festival
- handle NaN
- handle extra space at suffix
- change data type

In [62]:
df['Festival'].unique()

array(['No ', 'Yes ', 'NaN '], dtype=object)

In [63]:
df[df['Festival'] == 'NaN '].shape[0]

228

<font color = 'orange'>

#### <b> multiple_deliveries
- handle NaN
- change data type

In [59]:
df['multiple_deliveries'].unique()

array(['0', '1', '3', 'NaN ', '2'], dtype=object)

In [60]:
df[df['multiple_deliveries'] == 'NaN '].shape[0]

993

<font color = 'orange'>

#### <b> Type_of_vehicle
- handle extra space at suffix
- change data type

In [56]:
df['Type_of_vehicle'].unique()

array(['motorcycle ', 'scooter ', 'electric_scooter ', 'bicycle '],
      dtype=object)

<font color = 'orange'>

#### <b> Type_of_order
- handle extra space at suffix
- change data type

In [53]:
df['Type_of_order'].unique()

array(['Snack ', 'Drinks ', 'Buffet ', 'Meal '], dtype=object)

<font color = 'orange'>

#### <b> Road_traffic_density
- handle NaN
- handle extra space at suffix
- change data type

In [49]:
df['Road_traffic_density'].unique()

array(['High ', 'Jam ', 'Low ', 'Medium ', 'NaN '], dtype=object)

In [50]:
df[df['Road_traffic_density'] == 'NaN '].shape[0]

601

<font color = 'orange'>

#### <b> Delivery_person_Rattings
- handle NaN
- change data type

In [42]:
df['Delivery_person_Ratings'].unique()

array(['4.9', '4.5', '4.4', '4.7', '4.6', '4.8', '4.2', '4.3', '4', '4.1',
       '5', '3.5', 'NaN ', '3.8', '3.9', '3.7', '2.6', '2.5', '3.6',
       '3.1', '2.7', '1', '3.2', '3.3', '6', '3.4', '2.8', '2.9', '3'],
      dtype=object)

In [43]:
df[df['Delivery_person_Ratings'] == 'NaN '].shape[0]

1908

<font color = 'orange'>

#### <b> Delivery_person_Age
- handle NaN
- change data type

In [26]:
df['Delivery_person_Age'].unique()

array(['37', '34', '23', '38', '32', '22', '33', '35', '36', '21', '24',
       '29', '25', '31', '27', '26', '20', 'NaN ', '28', '39', '30', '15',
       '50'], dtype=object)

In [35]:
# 'NaN ' is present in Delivery_person_Age column - 1854
df[df['Delivery_person_Age'] == 'NaN '].shape[0]

1854

<font color = 'orange'>

#### <b> Weatherconditions
- handle Nan
- change data type

In [38]:
df['Weatherconditions'].unique()

array(['conditions Sunny', 'conditions Stormy', 'conditions Sandstorms',
       'conditions Cloudy', 'conditions Fog', 'conditions Windy',
       'conditions NaN'], dtype=object)

In [40]:
df[df['Weatherconditions'] == 'conditions NaN'].shape[0]

616

# Feature Engineering

"restaurant - delivery location" latitude & longitude
- Euclidean dist
- haversine dist
- manhattan dist
- estimate delivery time (assume 30 kmph)

time ordered & time order picked
- time lag
- ordered picked time ratio

order date
- hour
- day of week
- month
- quarter
- year